# A notebook with the FRB/US model
This notebook is based on the the FRB/US model packages located here (https://www.federalreserve.gov/econres/us-models-package.htm). The packages provides an Eviews (https://www.eviews.com/) implementation. The Eviews modelspecification has been transformed to ModelFlow BusinessLogic specification. 

In this notebook you can make your own experiments by updating variables. Also You can watch some of the capabilities of ModelFlow

FRB/US comes in two flawors: **VAR** in which expectaions are generated by backward looking equations, and **MCE** in which expectation are based on leaded values. 
This notebook implement the **VAR** flavour. 

The tool used is highly experimental and the results are purely for illustration. 

You can run each cell by pressing the run tool or shift+enter
To run all cells use select cell/run all in the menu bar. 

# Import Python libraries

In [1]:
%matplotlib inline

import pandas as pd
from numba import jit

from modelclass import model  
import modelmf

# Load the model and the data 
The model and the data has been prepared in a preprocessor step and has been wrapped into one file. 

In [3]:
mfrbus, basedf =  model.modelload('mfrbusvar.pcim')

res = mfrbus(basedf,'2020q1','2025q4',silent=1,stats=0)

Open file from URL:  https://raw.githubusercontent.com/IbHansen/modelflow-manual/main/model_repo/mfrbusvar.json


HTTPError: HTTP Error 404: Not Found

# Run alternative 

In [ ]:
# Make a new DataFrame
altdf = basedf.copy()

# Update variables in the dataframe
altdf=altdf.mfcalc('''\
rffintay_aerr = rffintay_aerr - 0.01
dmpex    = 0 
dmprr    = 0
dmptay   = 0
dmptlr   = 0 
dmpintay = 1
dmpalt   = 0
dmpgen   = 0

RFFMIN = 0.0
DMPTRSH = 0
''')

#Run the model 
with model.timer('Solve Frbus'):
    res2 = mfrbus(altdf,ldumpvar = 0,solver='newton',newton_absconv=0.0001)


In [ ]:
trans = {'XGDPN':'GDP real terms (XGDPN)','RFF':'Federal Funds Rate (RFF)','GFSRPN':'Federal Budget surplus',
        'RFFMIN':'Minimum Fed Funds'}

# Show the results 

In [ ]:
_ = mfrbus['XGDPN RFF'].dif.rename(trans).plot(colrow=1,sharey=0)

# Some interfaces 
. (dot) makes it easy to look at single variable. 

\[ \] (Slice) alows inspection of many variables selected with wildcars

# How is the interest rate determined
Below is a graph of the variables up- and downstream of the federal funds rate (RFF)

 - **des** determins if the descriptions is added to the variable name
 - **up and down**  sets the number of levels to show. To many makes it hard to use.
 - **png** if True the graph is displayed in png file. Some browsers prefer this. If set to 0 SVG output will be used. SVG can be zoomed without loss of details 
 - **pdf** if True a pdf file with the graph will be displayed. 
 - **HR** determins if the is a horisontal or vertical graph
 - **last** the values of a variable for the most recent run is displayed
 - **all** the values of a variable for the baseline and the most recent run is displayed

In [ ]:
with mfrbus.set_smpl('2020q1','2020q4'):
    mfrbus.RFF.draw(up=2,down=1,HR=0,png=0,svg=1,last=1,des=1)

# Lets look at how each variable contributes to changes in the equation

In [ ]:
with mfrbus.set_smpl('2020q1','2020q4'):
    mfrbus.rffintay.dekomp();

# We can look at attribution going back in the model 
Mouse-over will display the variable description

In [ ]:
with mfrbus.set_smpl('2020q1','2021q4'):
     mfrbus.rfftay.explain(up=1,svg=1,pdf=0);

# Print out values from the solution 

In [ ]:
mfrbus['rffta* rffrule*']

# Plot the values from the solution and the baseline

In [ ]:
_ = mfrbus['rffta* rffrule*'].dif.plot(colrow=1,sharey=0)

# Look at the model structure 

In [ ]:
if 0:  # set to 1 to get a dependency graph
    mfrbus.drawendo(lag=0,title='FRB/US endogeneous interpendencies',svg=1,browser=1)
# you will find the output files in different format in the /graph folder (graph/)

In [ ]:
fig   = mfrbus.plotadjacency(size=(40,40),title= 'Structure for the currrent time frame - used when solving')
fig.savefig('graph/FRB-US causality structure current time.pdf')

In [ ]:
fig   = mfrbus.plotadjacency(size=(40,40),nolag=True,title='Structure when we look across lags')
fig.savefig('graph/FRB-US causality structure across lags.pdf')